In [ ]:
import numpy as np
import pandas as pd

#import scipy
import matplotlib.pyplot as plt
#import os
#from scipy import io

import seaborn as sns
#import feather

#import pysubgroup as ps

#from scipy import stats
#from scipy.stats import zscore
#from sklearn.preprocessing import StandardScaler
#from sklearn import preprocessing
#from sklearn.model_selection import train_test_split

#from sklearn.cluster import KMeans
#from sklearn import preprocessing

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Read Data
df_bkpf = pd.read_excel('../Data1/BKPF.XLSX')

In [ ]:
df_bkpf.head(4)

In [ ]:
df_bkpf.BLART.unique()

In [ ]:
df_bseg = pd.read_feather('../Data1/BSEG.ftr')

In [ ]:
print(df_bseg.shape)
print(df_bkpf.shape)

In [ ]:
print(df_bseg.shape)
print(df_bkpf.shape)

In [ ]:
df_bseg.head(4)

# Merge

In [ ]:

#df_bseg.EBELN.value_counts()

In [ ]:
df_bk = pd.merge(df_bseg, df_bkpf, how='left', on='BELNR')

# check 

In [ ]:
df_bk.shape

In [ ]:
df_bk.loc[(df_bk['BLART'] == 'RE')].shape

In [ ]:
df_bk.loc[(df_bk['BSCHL'] == 31)].shape

In [ ]:
df_bk.loc[df_bk['BLART'] == 'RE'].BSCHL.unique()

In [ ]:
df_bk.loc[df_bk['BSCHL'] == 21].head()

In [ ]:
df_bk.loc[df_bk['BSCHL'] == 31].head(10)

In [ ]:
#sanity check
df_bk.loc[df_bk['EBELN'].isnull() == False].shape

## select only needed entries

In [ ]:
#df_bk = df_bk.loc[(df_bk['BSCHL'] == 21) | (df_bk['BSCHL'] == 31)].copy() #21
df_bk = df_bk.loc[(df_bk['BSCHL'] == 31)].copy()
df_bk.shape

In [ ]:
df_bk.head()

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_bk)

# Data Cleansing

## initial cleaning

In [ ]:
df = df_bk.copy()
print(df.shape)

In [ ]:
df['TMSTMP'] = pd.to_datetime(df['CPUDT'].astype(str) + df['CPUTM'].astype(str) , format='%Y-%m-%d%H:%M:%S')

In [ ]:
#df['HZUON'] = df['HZUON'].astype(int)

In [ ]:
#df['HZUON'] = pd.to_datetime(df['HZUON'].astype(str).str[:8] , format='%Y%m%d')
#df['SAKNR'] = df['SAKNR'].astype(int)
df['LOKKT'] = df['LOKKT'].astype(int)


In [ ]:
# Spalten die nur NAN enthalten entfernen
df = df.dropna(axis=1, how='all')

# Spalten die ausschließlich XY Werte enthalten (0 / X / 1 / etc.) dropen
df = df.loc[:, (df.astype(str) != 0).any(axis=0)]
df = df.loc[:, (df.astype(str) != '0').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,0').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,00').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,000').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0.0').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0.00').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0.000').any(axis=0)]
df = df.loc[:, (df.astype(str) != 'X').any(axis=0)]
df = df.loc[:, (df.astype(str) != '1').any(axis=0)]


# spaletn entfernen die immer die selben Werte enthalten
keep_columns = [col for col in df.columns if len(df[col].unique()) > 1]
df = df[keep_columns].copy()

#print(len(df_io.index))
print(df.shape)

In [ ]:
df.columns

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
# Delete BUKRs etc.

In [ ]:
df.drop(['BUKRS_x','BUKRS_y','KOKRS'],axis=1, inplace=True)

## data types correction

In [ ]:
df.describe(include='all').T

In [ ]:
for (columnName, columnData) in df.items():
    print('Colunm Name : ', columnName )
    print('Colunm Name : ', df[columnName].dtypes )
    print('Number of unique values : ', df[columnName].nunique() )
    print('Column unique values : ', np.unique(df[columnName].astype(str)) )
    print('######')

In [ ]:
df.sample(4)

In [ ]:
df.dtypes

In [ ]:
cont_Cols = ['BELNR', 'ZUONR', 'XBLNR', 'AWKEY']

cat_Cols = []

num_Cols = ['DMBTR', 'WRBTR', 'PSWBT', ] #'MENGE' 'ERFMG' 'BPMNG'

dat_Cols = ['BUDAT', 'CPUTM', 'CPUDT', 'WWERT', 'BLDAT', 'TMSTMP', 'VALUT', 'HZUON'] #HZUON ZFBDT

In [ ]:
#determin cat cols
#cat cols are all cols - the cols we already defined as num / dat / cat
unw = num_Cols + dat_Cols + cont_Cols
list1 = df.columns.to_list()
cat_Cols = [x for x in list1 if x not in unw]

In [ ]:
#sanity check of cols
print(len(cont_Cols)+len(cat_Cols)+len(num_Cols)+len(dat_Cols))
print(df.shape[1]) 
print(cat_Cols)

In [ ]:
# Change dtype to Category for relevant features
df_bk_1 = df.copy()
df_bk_1.head()

# we have to change NaN top 0 to avoid the infer dtype "object" for some cols
df_bk_1['ZUONR'] = df_bk_1['ZUONR'].fillna(0)
#df_bk_1['BPRME'] = df_bk_1['BPRME'].fillna(0)  

# change Dtype to Cat for categorical columns
cats_cat = df_bk_1[cat_Cols]


for (columnName, columnData) in cats_cat.items():
    df_bk_1[columnName] = df_bk_1[columnName].astype(str)
    if isinstance(df_bk_1[columnName], object):
        s = pd.Series(df_bk_1[columnName], dtype="category")
        df_bk_1[columnName] = df_bk_1[columnName].astype('category')


In [ ]:
#check 1
df_bk_1.shape

In [ ]:
df_bk_1.head()

In [ ]:
#check 2
print('Categorical columns: {}'.format(list(df_bk_1.select_dtypes('category').columns)))

In [ ]:
#check 3
df_bk_1.shape

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_bk_1)

In [ ]:
df_bk_1.to_feather('../Data/D3_BKPF-BSEG_raw.ftr')

# lets check the dataset properities / Correlation  / distribution

In [ ]:
# Compute the corr matrix, only for numerics since we use category dtype
corr = df_bk_1.corr(numeric_only=True)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(10, 10))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
df_bk_1.sample(10)

#manually delete the hihgly correlating features in both dataframes
# WRBTR PSWBT VBEL2 POSN2 ERFMG AUFPL SAKNR SAKNR LOKKT KIDNO
df_bk_1 = df_bk_1.drop(['WRBTR'], axis=1)
df_bk_1 = df_bk_1.drop(['PSWBT'], axis=1)
df_bk_1 = df_bk_1.drop(['VBEL2'], axis=1)
df_bk_1 = df_bk_1.drop(['POSN2'], axis=1)
df_bk_1 = df_bk_1.drop(['ERFMG'], axis=1)
df_bk_1 = df_bk_1.drop(['AUFPL'], axis=1)
df_bk_1 = df_bk_1.drop(['SAKNR'], axis=1)
df_bk_1 = df_bk_1.drop(['LOKKT'], axis=1)
df_bk_1 = df_bk_1.drop(['KIDNO'], axis=1)

df = df.drop(['WRBTR'], axis=1)
df = df.drop(['PSWBT'], axis=1)
df = df.drop(['VBEL2'], axis=1)
df = df.drop(['POSN2'], axis=1)
df = df.drop(['ERFMG'], axis=1)
df = df.drop(['AUFPL'], axis=1)
df = df.drop(['SAKNR'], axis=1)
df = df.drop(['LOKKT'], axis=1)
df = df.drop(['KIDNO'], axis=1)

In [ ]:

# Compute the corr matrix and 
corr = df_bk_1.corr(numeric_only=True)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(7, 7))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
df_bk_1.shape

In [ ]:
df_bk_1.to_feather('../Data/D3_BKPF-BSEG_cleaned.ftr')